## Desenvolvimento do modelo de predição de preço

Exploração inicial de modelos de base

In [1]:
import pandas as pd
from dagshub.data_engine import datasources
import mlflow
import dagshub
from sklearn.model_selection import train_test_split
import mlflow.sklearn
import mlflow.catboost
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import mlflow.models.signature
from mlflow.models import infer_signature
from catboost import CatBoostRegressor

In [2]:
ds = datasources.get_datasource("pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl","processed")

Accessing as pedromonnt

In [3]:
ds.all().dataframe

Output()

,path,datapoint_id,dagshub_download_url,media type,size
0,laptop-price-brl-processed.csv,102408450,https://dagshub.com/api/v1/repos/pedromonnt/fi...,text/plain,52559


In [4]:
res = ds.head()

for dp in res:
    dataset_url = dp.download_url

Output()

In [5]:
dataset_url

'https://dagshub.com/api/v1/repos/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl/raw/main/data/processed/laptop-price-brl-processed.csv'

In [6]:
df = pd.read_csv(dataset_url)
df.head()

,ram_gb,ssd,hdd,graphic_card_gb,warranty,price,brand_asus,brand_dell,brand_hp,brand_lenovo,...,ram_type_other,os_other,os_windows,os_bit_32-bit,os_bit_64-bit,weight_casual,weight_gaming,weight_thinnlight,touchscreen_0,touchscreen_1
0,4,0,1024,0,0,2321,1,0,0,0,...,0,0,1,0,1,1,0,0,1,0
1,4,0,1024,0,0,2613,0,0,0,1,...,0,0,1,0,1,1,0,0,1,0
2,4,0,1024,0,0,2680,0,0,0,1,...,0,0,1,0,1,1,0,0,1,0
3,8,512,0,2,0,4689,1,0,0,0,...,0,0,1,1,0,1,0,0,1,0
4,4,0,512,0,0,1808,1,0,0,0,...,0,0,1,0,1,1,0,0,1,0


## Desenvolvimento e experimentos de modelos

In [7]:
dagshub.init(repo_owner="pedromonnt", repo_name="fiap-ds-mlops-10dtsr-laptop-pricing-brl", mlflow = True)

Initialized MLflow to track repo "pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl"

Repository pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl initialized!

In [8]:
mlflow.autolog()

2025/07/10 00:04:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/07/10 00:04:17 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/10 00:04:17 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [9]:
features = list(df.columns)
features.remove("price")

In [10]:
features

['ram_gb',
 'ssd',
 'hdd',
 'graphic_card_gb',
 'warranty',
 'brand_asus',
 'brand_dell',
 'brand_hp',
 'brand_lenovo',
 'brand_other',
 'processor_brand_amd',
 'processor_brand_intel',
 'processor_brand_m1',
 'processor_name_core i3',
 'processor_name_core i5',
 'processor_name_core i7',
 'processor_name_other',
 'processor_name_ryzen 5',
 'processor_name_ryzen 7',
 'ram_type_ddr4',
 'ram_type_other',
 'os_other',
 'os_windows',
 'os_bit_32-bit',
 'os_bit_64-bit',
 'weight_casual',
 'weight_gaming',
 'weight_thinnlight',
 'touchscreen_0',
 'touchscreen_1']

In [11]:
X = df[features]

In [12]:
len(features)

30

In [13]:
y = df["price"]
y

0       2321
1       2613
2       2680
3       4689
4       1808
       ...  
775     9111
776     9714
777    10049
778     9580
779     3852
Name: price, Length: 780, dtype: int64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [15]:
def evaluate_and_log_model(kind, model_name, model, X_test, y_test):
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    mape = mean_absolute_percentage_error(y_test, predictions)
    
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("MAPE", mape)

    signature = infer_signature(X_test, predictions)

    if kind == "catboost":
        mlflow.catboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "xgboost":
        mlflow.xgboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "lightgbm":
        mlflow.lightgbm.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    else:
        mlflow.sklearn.log_model(model, model_name, signature=signature, input_example=X_test[:5])

    print(f"Model {model_name} logged with MSE: {mse}, MAE: {mae}, R2: {r2}, MAPE: {mape}")

### Experimento com Ridge Regression

In [ ]:
with mlflow.start_run(run_name = "Ridge Regression"):
    param_grid = {
        'alpha': [0.1, 1.0, 10.0, 100.0],
        'fit_intercept': [True, False]
    }

    ridge = Ridge()

    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better = False))
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    mlflow.log_param("best_alpha", grid_search.best_params_['alpha'])
    mlflow.log_param("best_fit_intercept", grid_search.best_params_['fit_intercept'])

    evaluate_and_log_model("sklearn", "ridge_regression", best_model, X_test, y_test)

2025/07/10 00:04:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\pedro\Downloads\fiap-ds-mlops-10dtsr-laptop-pricing-brl\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/10 00:04:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\

🏃 View run salty-squid-123 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/8daa17c861ff4638b6f6a3b4344a189c
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run learned-bird-88 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/08dc8a48c21c4321baa0ccf6e82fc3c2
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run puzzled-ape-742 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/b47fa51c03434c5abfe550669bb1b914
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run brawny-crane-492 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/452dc523882349099aab3fcb0a06afab

c:\Users\pedro\Downloads\fiap-ds-mlops-10dtsr-laptop-pricing-brl\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Model ridge_regression logged with MSE: 3024089.113899016, MAE: 1137.7871730265826, R2: 0.6421034856259351, MAPE: 0.22065408855367982
🏃 View run Ridge Regression at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/ca88768ffec546989150e4d7d889c411
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0


🏃 View run delicate-penguin-723 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/de904667f79e432da30da8f52d56455c
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run receptive-whale-808 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/785c1af6e9e34c87a84407eb82733024
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run mercurial-cod-117 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/93c52fbce9474b08bb240a2952b0b361
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run fearless-grub-72 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/02840695471245f4ac452

### Decision Tree Regressor

In [17]:
with mlflow.start_run(run_name = "Decision Tree Regression"):
    param_grid = {
        'max_depth': [None, 3, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    ridge = DecisionTreeRegressor()

    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better = False))
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])

    evaluate_and_log_model("sklearn", "decision_tree_regression", best_model, X_test, y_test)

2025/07/10 00:05:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\pedro\Downloads\fiap-ds-mlops-10dtsr-laptop-pricing-brl\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/10 00:05:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\

Model decision_tree_regression logged with MSE: 3698554.4767268803, MAE: 1047.0384696152662, R2: 0.5622814984653437, MAPE: 0.1664400851319324
🏃 View run Decision Tree Regression at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/f125de6577ee45e6bf2a53bfbaad1f97
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0


## XGBoost

In [18]:
with mlflow.start_run(run_name = "XGBoost Regressor Advanced"):
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'gamma': [0, 1],
        'reg_alpha': [0, 0.1],
        'reg_lambda': [1, 5],
        'min_child_weight': [1, 3]
    }

    xgb = XGBRegressor(random_state = 42, verbosity = 0)

    grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better = False))
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    mlflow.log_param("best_n_estimators", best_model.n_estimators)
    mlflow.log_param("best_max_depth", best_model.max_depth)
    mlflow.log_param("best_learning_rate", best_model.learning_rate)

    evaluate_and_log_model("xgboost", "XGBoost Regressor", best_model, X_test, y_test)

2025/07/10 00:06:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\pedro\Downloads\fiap-ds-mlops-10dtsr-laptop-pricing-brl\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/10 00:16:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\

Model XGBoost Regressor logged with MSE: 2602369.0, MAE: 871.9007568359375, R2: 0.6920134425163269, MAPE: 0.13540245592594147
🏃 View run XGBoost Regressor Advanced at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/716800d463e44835b8b38af5a53fbd44
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0


### Registro de modelo em produção

In [19]:
run_id = "9bc012b5212d41709b7c2626cc55f498"
mlflow.register_model(model_uri=f"runs:/{run_id}/model", name = "laptop-pricing-model-brl")

Registered model 'laptop-pricing-model-brl' already exists. Creating a new version of this model...
2025/07/10 00:17:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: laptop-pricing-model-brl, version 3
Created version '3' of model 'laptop-pricing-model-brl'.


<ModelVersion: aliases=[], creation_timestamp=1752117453608, current_stage='None', description='', last_updated_timestamp=1752117453608, name='laptop-pricing-model-brl', run_id='9bc012b5212d41709b7c2626cc55f498', run_link='', source='mlflow-artifacts:/0ed9d89ca3b4429a81acfe440280248d/9bc012b5212d41709b7c2626cc55f498/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='3'>